In [3]:
import os
import pandas as pd
from tqdm import tqdm
import pickle


In [4]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")
filename = "europarl-v7.da-en.da"
with open(filename, "r", encoding="UTF-8") as file:
    lines = file.readlines()
with open("nutids_r.pickle", "rb") as f:
    nutids_r = pickle.load(f)
with open("nutids_r_stem.pickle", "rb") as f:
    nutids_r_stem = pickle.load(f)

In [13]:
import random

step_rates = [1, 2, 3]
pre_pad_result = []
len_of_list = int(len(lines))
for i in tqdm(range(0, len_of_list//3,step_rates[0])):
    pre_pad_result.append((' '.join(lines[i:i+step_rates[0]])).split())
for i in tqdm(range(len_of_list//3, 2*len_of_list//3,step_rates[1])):
    pre_pad_result.append((' '.join(lines[i:i+step_rates[1]])).split())

random.shuffle(pre_pad_result)

100%|██████████| 328134/328134 [00:27<00:00, 11830.82it/s] 


In [14]:
scope = 10
middle = int(scope/2)
padding = int(scope/2)

In [15]:
pre_cleaned_result = [["<PAD>"]*padding + lst + ["<PAD>"]*padding for lst in tqdm(pre_pad_result)]

100%|██████████| 984400/984400 [00:39<00:00, 24998.00it/s] 


In [16]:
char = "*@;:!\"?«».,"
post_pad_result = [[word.translate(str.maketrans('', '', ''.join(char))) for word in pre_cleaned_lst] for pre_cleaned_lst in tqdm(pre_cleaned_result)]

# for i in tqdm(range(len(pre_cleaned_result))):
#     pre_cleaned_lst = pre_cleaned_result[i]
#     lst = [word.translate(str.maketrans('', '', ''.join(char))) for word in pre_cleaned_lst]
#     post_pad_result.append(lst)

100%|██████████| 984400/984400 [01:21<00:00, 12106.52it/s]


In [17]:
saved_post_pad_result = post_pad_result

In [18]:
post_pad_result = saved_post_pad_result[:50000]

In [19]:
bøjning = {0: "infinitiv", 1: "førnutidsformen"}
half_scope = int(scope/2)
big_lst = []
output_lst = []


for i in tqdm(range(len(post_pad_result))):
    current_lst = post_pad_result[i]
    for x in range(len(current_lst)):
        current_word = current_lst[x]
        try: stemmed_word = nutids_r_stem[current_word]
        except: continue
        current_dataset = current_lst[x-half_scope:x+half_scope+1]
        try: current_dataset[half_scope] = stemmed_word
        except: print(current_dataset); continue
        big_lst.append((" ".join(current_dataset)).lower())
        if nutids_r[stemmed_word][0] == current_word:
            output = 0
        elif nutids_r[stemmed_word][1] == current_word:
            output = 1
        else:
            print("ERROR")
        output_lst.append(output)


100%|██████████| 50000/50000 [00:01<00:00, 37801.45it/s]


In [20]:
df = pd.DataFrame()
df["comment_text"] = big_lst
df["label"] = output_lst

print(len(df))
df = df[:6000000]

def distribution(df):
    print(df["label"].value_counts())

distribution(df)
print(len(df))
header = ["comment_text", "label"]
df.to_csv("../Datasets/EuroparlNutidsr.csv", encoding="UTF-8", index=False, sep=";")

143961
0    92156
1    51805
Name: label, dtype: int64
143961


--- 
### Making testset for nutids-r module

In [5]:
import os
import pandas as pd
from tqdm import tqdm
import pickle

In [6]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")
filename = "europarl-v7.da-en.da"
with open(filename, "r", encoding="UTF-8") as file:
    lines = file.readlines()
with open("nutids_r_bøjninger.pickle", "rb") as f:
    nutids_r_bøjninger = pickle.load(f)
with open("nutids_r_stem.pickle", "rb") as f:
    nutids_r_stem = pickle.load(f)

In [12]:
df = pd.read_csv("../Datasets/EuroparlNutidsr_testset.csv", encoding="UTF-8", sep=";")
correct_sentences = list(df["correct"])
wrong_sentences = list(df["wrong"])
from IPython.display import clear_output

def save_df():
    df = pd.DataFrame()
    df["wrong"] = wrong_sentences
    df["correct"] = correct_sentences
    df.to_csv("../Datasets/EuroparlNutidsr_testset.csv", encoding="UTF-8", index=False, sep=";")

for line in tqdm(lines[122:1000]):
    line = line.strip("\n")
    words = line.split()
    for i, word in enumerate(words):
        print("hey")
        symbol = ""
        if word[-1] in ".,?!":
            symbol = word[-1]
            word = word[:-1]
        try: stemmed = nutids_r_stem[word]
        except: continue
        if word[-1] == "s":
            continue
        print(line)
        if nutids_r_bøjninger[stemmed][0] == word:
            bøjet_ord = nutids_r_bøjninger[stemmed][1]
        else:
            bøjet_ord = nutids_r_bøjninger[stemmed][0]
        print(f"{word} -> {bøjet_ord}")
        to_continue = int(input(f"1 for Ja, 2 for Nej."))
        clear_output()
        if to_continue == 2:
            continue
        words[i] = bøjet_ord + symbol
    new_line = " ".join(words)
    wrong_sentences.append(new_line)
    correct_sentences.append(line)
    save_df()

  2%|▏         | 16/894 [02:18<1:40:50,  6.89s/it]

hey
hey
hey
hey
hey
hey
hey
Fru formand, da jeg kan høre en smule latter fra Socialdemokraterne - jeg har fået fortalt, at brede kredse i Den Socialdemokratiske Gruppe også gerne vil have taget dette punkt af dagsordenen, fordi der ved afstemningen på Formandskonferencen ikke forelå noget votum fra arbejdsgruppen af ansvarlige kolleger i Den Socialdemokratiske Gruppe.
kan -> kunne


  2%|▏         | 16/894 [02:20<2:08:02,  8.75s/it]


ValueError: invalid literal for int() with base 10: ''

----

In [55]:
import os
import pandas as pd
from tqdm import tqdm
import pickle

In [56]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")
filename = "europarl-v7.da-en.da"
with open(filename, "r", encoding="UTF-8") as file:
    lines = file.readlines()
with open("nutids_r_bøjninger.pickle", "rb") as f:
    nutids_r_bøjninger = pickle.load(f)
with open("nutids_r_stem.pickle", "rb") as f:
    nutids_r_stem = pickle.load(f)

In [57]:
# If changed all of the dataset also needs to be changed

padding_left = 10
padding_right = 4

scope = 10
padding = int(scope/2)


In [58]:
testset = []
labels = []

In [59]:
df = pd.read_csv("../Datasets/EuroparlSentToPos.csv", encoding="UTF-8", sep=";")
sentences = list(df["sent"])
all_pos = list(df["pos"])
df

,sent,pos
0,Genoptagelse af sessionen,NOUN ADP NOUN
1,"Jeg erklærer Europa-Parlamentets session, der ...",PRON VERB NOUN PUNCT PRON AUX VERB NOUN DET AD...
2,"Som De kan se, indfandt det store ""år 2000-pro...",ADP PRON AUX VERB VERB DET ADJ NOUN X PRON PUN...
3,De har udtrykt ønske om en debat om dette emne...,PRON AUX VERB NOUN ADP DET NOUN ADP DET NOUN A...
4,I mellemtiden ønsker jeg - som også en del kol...,ADP NOUN VERB PRON PUNCT ADP ADV DET NOUN NOUN...
...,...,...
219995,(Bifald),PUNCT
219996,Lad mig komme med endnu en bemærkning om konve...,VERB PRON VERB ADP ADV DET NOUN ADP NOUN
219997,"Vi knytter store håb til det, De beslutter i L...",PRON VERB ADJ NOUN ADP PRON PRON VERB ADP PROPN
219998,Den talmæssige sammensætning af konventet er k...,DET ADJ NOUN ADP NOUN VERB PUNCT


In [60]:
def get_pos_tags(index):
    return all_pos[index].split()

for i in tqdm(range(len(sentences))):
    line = sentences[i]
    if len(str(line)) < 1 or str(line) == "nan":
        continue
    line = line.strip("\n")
    true_words = line.split()
    pos = get_pos_tags(i)
    words = ["<PAD>"]*padding_left + pos + ["<PAD>"]*padding_right
    for i, word in enumerate(true_words):
        try: stemmed = nutids_r_stem[word]
        except: continue
        if word[-1] == "s":
            continue
        if nutids_r_bøjninger[stemmed][0] == word:
            labels.append(1)
        else:
            labels.append(0)
        testset.append(" ".join(words[i:i+padding_left+padding_right+1]))

100%|██████████| 220000/220000 [00:04<00:00, 49277.17it/s]


In [61]:
df = pd.DataFrame(zip(testset, labels), columns=["comment_text", "label"])

In [62]:
df

,comment_text,label
0,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
1,DET ADJ NOUN ADP PUNCT ADV DET NOUN AUX PRON V...,1
2,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
3,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> PRON...,1
4,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
...,...,...
488318,PUNCT ADV SCONJ DET NUM NOUN AUX ADJ PUNCT PRO...,0
488319,AUX ADJ PUNCT PRON VERB ADP PUNCT CCONJ ADP PU...,0
488320,PUNCT PRON VERB ADP PUNCT CCONJ ADP PUNCT SCON...,0
488321,PUNCT CCONJ ADP PUNCT SCONJ PRON VERB DET NOUN...,0


In [64]:
comment_text = df["comment_text"].values
mistakes = [l for l in comment_text if not l.isupper()]
middle = [l.split()[padding_left] for l in comment_text]

from collections import Counter

counter = Counter(middle)
for key, value in counter.items():
    print(key, ": ", value)

VERB :  300197
AUX :  53671
NOUN :  37439
PRON :  45215
ADJ :  18992
ADV :  11994
SCONJ :  9413
NUM :  5019
ADP :  6280
X :  99
INTJ :  2
PROPN :  2


In [65]:
df.to_csv("../Datasets/EuroparlNutidsr_trainset.csv", encoding="UTF-8", index=False, sep=";")

In [67]:
verb_text_and_labels = [(c, l) for c, l in zip(comment_text, labels) if c.split()[padding_left] == "VERB"]
verb_text = [c for c, l in verb_text_and_labels]
verb_labels = [l for c, l in verb_text_and_labels]

df = pd.DataFrame(zip(verb_text, verb_labels), columns=["comment_text", "label"])
df.to_csv("../Datasets/EuroparlNutidsr_trainset_verbs.csv", encoding="UTF-8", index=False, sep=";")
df

,comment_text,label
0,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
1,DET ADJ NOUN ADP PUNCT ADV DET NOUN AUX PRON V...,1
2,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
3,ADV DET NOUN NOUN AUX VERB ADP PUNCT SCONJ PRO...,0
4,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
...,...,...
300192,<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...,0
300193,<PAD> <PAD> <PAD> PRON VERB NOUN NOUN NOUN ADP...,1
300194,PUNCT ADV SCONJ DET NUM NOUN AUX ADJ PUNCT PRO...,0
300195,PUNCT PRON VERB ADP PUNCT CCONJ ADP PUNCT SCON...,0


In [74]:
counts = {i: 0 for i in range(14)}
for sentence in verb_text:
    num_pads = sentence.count("<PAD>")
    counts[num_pads] += 1

from tabulate import tabulate

table = [["Number of <PAD> occurrences", "Number of sentences"]]
for i in range(14):
    table.append([i, counts[i]])

print(tabulate(table, headers="firstrow", tablefmt="github"))

|   Number of <PAD> occurrences |   Number of sentences |
|-------------------------------|-----------------------|
|                             0 |                138539 |
|                             1 |                 21067 |
|                             2 |                 21729 |
|                             3 |                 21136 |
|                             4 |                 12495 |
|                             5 |                 13207 |
|                             6 |                 12222 |
|                             7 |                 16923 |
|                             8 |                 14211 |
|                             9 |                 26631 |
|                            10 |                  1229 |
|                            11 |                   606 |
|                            12 |                   201 |
|                            13 |                     1 |
